In [9]:
from alpha_vantage.fundamentaldata import FundamentalData
import pandas as pd
import time
import sqlite3

In [10]:
# apikey = '4YLKM5SFV5RXMQCG'
# fd = FundamentalData(apikey, output_format = 'pandas')

# symbol = 'ACAD'

# data = fd.get_balance_sheet_quarterly(symbol)
# data1 = fd.get_income_statement_quarterly(symbol)
# data = data[0].T
# data1 = data1[0].T

In [11]:
# result = pd.concat([data, data1])
# result = result.T.reset_index()
# new_result = result.loc[:,~result.columns.duplicated(keep='first')]
# new_result = new_result.drop(new_result.columns[[0,2]], axis=1)

# new_result['netIncome'] = result['netIncome']
# new_result.iloc[:, 1:] = new_result.iloc[:, 1:].apply(lambda x: pd.to_numeric(x, errors='coerce')).astype(float)

# Equity = (new_result['totalAssets'] - new_result['totalLiabilities'])
# NetProfit = (new_result['grossProfit'] - new_result['operatingExpenses'])
# ROA_value = (new_result['netIncome'] / new_result['totalAssets']) * 100
# ROE_value = (new_result['netIncome'] / new_result['totalShareholderEquity']) * 100

# new_result['Equity'] = Equity
# new_result['NetProfit'] = NetProfit
# new_result['ROA'] = ROA_value
# new_result['ROE'] = ROE_value
# new_result['Symbol'] = symbol

In [12]:
# result_after = new_result.drop(new_result.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,28,29,30,
#                                           31,32,33,34,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]]
#                                           , axis=1)
# result_after = result_after.round(3)
# result_after = result_after.rename(columns={"fiscalDateEnding": "Period", "totalAssets": "Asset", "totalLiabilities": "TotalDebt",
#                              "totalRevenue": "Revenue"})
# result_after = result_after[['Asset', 'TotalDebt', 'Equity', 'Revenue', 'NetProfit', 'ROA', 'ROE', 'Period', 'Symbol']]
# result_after['Period'] = pd.to_datetime(result_after['Period'])
# result_after = result_after[~(result_after['Period'].isna())]
# result_after = result_after.sort_values(by=['Period'], ascending=True).reset_index(drop=True)
# result_after['Period'] = result_after['Period'].dt.quarter.astype(str) +'Q'+ result_after['Period'].dt.year.astype(str)

# result_after

In [13]:
# result_after.dtypes

In [14]:
def get_symbol_id(market):
        con = sqlite3.connect('share_V2.sqlite')
        cur = con.cursor()
        sql = f"""select Information.Symbol,Information.SymbolId  
                from Information Inner join Market on Market.MarketId = Information.MarketId
                where Market.Mname = "{market}"
                """
        cur.execute(sql)
        records = cur.fetchall()
        con.close()
        symbol_id = {}
        for share in records:
            symbol_id[share[0]] = share[1]
        return symbol_id

In [15]:
def alphaVantage_data(symbol):
    apikey = '4YLKM5SFV5RXMQCG'
    fd = FundamentalData(apikey, output_format = 'pandas')
    balance_sheet = fd.get_balance_sheet_quarterly(symbol)
    income_state = fd.get_income_statement_quarterly(symbol)
    balance_sheet = balance_sheet[0].T
    income_state = income_state[0].T
    return (balance_sheet, income_state)

def calInfo(data):
    Equity = (data['totalAssets'] - data['totalLiabilities'])
    NetProfit = (data['grossProfit'] - data['operatingExpenses'])
    ROA_value = (data['netIncome'] / data['totalAssets']) * 100
    ROE_value = (data['netIncome'] / data['totalShareholderEquity']) * 100
    data['Equity'] = Equity
    data['NetProfit'] = NetProfit
    data['ROA'] = ROA_value
    data['ROE'] = ROE_value
    return data

def arrangeAlphaVantageDetail(symbol):
    data_balance, data_income = alphaVantage_data(symbol)
    result = pd.concat([data_balance, data_income])
    result = result.T.reset_index()
    new_result = result.loc[:,~result.columns.duplicated(keep='first')]
    new_result = new_result.drop(new_result.columns[[0,2]], axis=1)
    new_result['netIncome'] = result['netIncome']
    new_result.iloc[:, 1:] = new_result.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
    new_result = calInfo(new_result)
    new_result['SymbolId'] = str(int(get_symbol_id("NASDAQ")[symbol]))
    result_after = new_result.drop(new_result.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,28,29,30,
                                          31,32,33,34,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]]
                                          , axis=1)
    result_after = result_after.round(3)
    result_after = result_after.rename(columns={"fiscalDateEnding": "Period", "totalAssets": "Asset", "totalLiabilities": "TotalDebt",
                                "totalRevenue": "Revenue"})
    result_after = result_after[['Asset', 'TotalDebt', 'Equity', 'Revenue', 'NetProfit', 'ROA', 'ROE', 'Period', 'SymbolId']]
    result_after['Period'] = pd.to_datetime(result_after['Period'])
    result_after = result_after[~(result_after['Period'].isna())]
    result_after = result_after.sort_values(by=['Period'], ascending=True).reset_index(drop=True)
    result_after['Period'] = result_after['Period'].dt.quarter.astype(str) +'Q'+ result_after['Period'].dt.year.astype(str)
    return result_after

In [16]:
result = arrangeAlphaVantageDetail('META')
result

C:\Users\Admin\AppData\Local\Temp\ipykernel_14596\2211674929.py:28: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_result.iloc[:, 1:] = new_result.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')


,Asset,TotalDebt,Equity,Revenue,NetProfit,ROA,ROE,Period,SymbolId
0,88945000000,11325000000,77620000000,11966000000,5280000000,5.608,6.426,1Q2018,3089
1,90291000000,10909000000,79382000000,13231000000,5705000000,5.655,6.432,2Q2018,3089
2,92452000000,12110000000,80342000000,13727000000,5625000000,5.556,6.394,3Q2018,3089
3,97334000000,13207000000,84127000000,16692000000,7364000000,7.070,8.180,4Q2018,3089
4,109477000000,22961000000,86516000000,15077000000,3161000000,2.219,2.808,1Q2019,3089
5,117006000000,28244000000,88762000000,16883000000,4420000000,2.236,2.947,2Q2019,3089
6,124418000000,30419000000,93999000000,17648000000,6985000000,4.896,6.480,3Q2019,3089
7,133376000000,32322000000,101054000000,20966000000,8582000000,5.510,7.272,4Q2019,3089
8,138371000000,33067000000,105304000000,17734000000,5522000000,3.543,4.655,1Q2020,3089
9,139691000000,29244000000,110447000000,18683000000,5781000000,3.707,4.688,2Q2020,3089


In [17]:
# def AlphaVantageDetail():
#     values = allSymbolnasdaq()[:50]
#     details_alpha = []
#     count = 0
#     for name in values:
#         if (count == 2):
#             time.sleep(65)
#             df = arrangeAlphaVantageDetail(name)
#             df_old = details_alpha[0]
#             df_new = pd.concat([df_old, df])
#             details_alpha[0] = df_new
#             count = 0
#         elif len(details_alpha) == 0:
#             df1 = arrangeAlphaVantageDetail(name)
#             details_alpha.append(df1)
#         else:
#             df2 = arrangeAlphaVantageDetail(name)
#             df_old = details_alpha[0]
#             df_new = pd.concat([df_old, df2])
#             details_alpha[0] = df_new
#         count+=1
#     return details_alpha[0]